# Bout, Stim, and Duration Post Hoc Analysis(CSV for Database) 

In [102]:
import joblib
import os
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import pickle
import time
from tkinter import *
import tkinter as tk
from tkinter import filedialog
from tqdm import tqdm
from tqdm import notebook
import itertools
import math
import seaborn as sns
import statistics 
from collections import OrderedDict
import csv

In [103]:
# Create Tk root
root = Tk()
# Hide the main window
root.withdraw()
root.call('wm', 'attributes', '.', '-topmost', True)
infiles = filedialog.askopenfilename(multiple=True, title='load posthoc-predictions.npy')

%gui tk

infiles

('Z:/KayCei/HDC/AH1-KO/120423-LIGHTS OFF/predict_video_exp_16.npy',)

In [104]:
# Check if the user selected any files
if infiles:
    # Assuming infiles is a list of selected file paths, let's use the first one
    selected_file = infiles[0]

    # Extract the directory path from the selected file's full path
    directory_path = os.path.dirname(selected_file)
    
#print(directory_path)

In [105]:
Random_List = [] #creates a array known as random list 
for f in range(len(infiles)):
    arr = np.load(infiles[f])
    Random_List.extend(arr) 
#print(Random_List)

In [106]:
behavior_labels = {
    0.0: "insignificant",
    1.0: "insignificant",
    2.0: "insignificant",
    3.0: "insignificant",
    4.0: "Rear",
    5.0: "insignificant",
    6.0: "insignificant",
    7.0: "Investigate Type 1",
    8.0: "Investigate Type 2", 
    9.0: "Contra-Itch",
    10.0: "Investigate Type 3",
    11.0: "insignificant",
    12.0: "insignificant",
    13.0: "insignificant",
    14.0: "Contra-Body Groom",
    15.0: "Face Groom Type 1",
    16.0: "Dive/Scrunch",
    17.0: "Head Groom",
    18.0: "Ipsi-Orient",
    19.0: "insignificant",
    20.0: "Face Groom Type 2",
    21.0: "Ipsi-Body Groom",
    22.0: "Ipsi-Itch Type 1",
    23.0: "insignificant",
    24.0: "insignificant",
    25.0: "Paw-Groom",
    26.0: "Locomotion",
    27.0: "insignificant",
    28.0: "Contra-Orient",
}

In [107]:
from collections import Counter

# Assuming Random_List contains behavior labels as values
label_counts = Counter(Random_List)

# Print the length of each behavior label
for label, count in label_counts.items():
    print(f"Label: {label}, Count: {count}")


Label: 26, Count: 13509
Label: 28, Count: 2211
Label: 18, Count: 2810
Label: 8, Count: 12643
Label: 27, Count: 678
Label: 3, Count: 940
Label: 15, Count: 342
Label: 7, Count: 1215
Label: 5, Count: 841
Label: 6, Count: 621
Label: 9, Count: 1136
Label: 16, Count: 341
Label: 22, Count: 237
Label: 23, Count: 303
Label: 10, Count: 591
Label: 21, Count: 203
Label: 25, Count: 23
Label: 24, Count: 64
Label: 17, Count: 74
Label: 14, Count: 224
Label: 4, Count: 55
Label: 1, Count: 54
Label: 19, Count: 116
Label: 11, Count: 30
Label: 13, Count: 33
Label: 20, Count: 4
Label: 12, Count: 19
Label: 0, Count: 13
Label: 2, Count: 1


# Total Time of Video in deciseconds

In [108]:
duration_in_deciseconds = len(Random_List)
duration_in_minutes = duration_in_deciseconds / 600
duration_in_frames = duration_in_deciseconds * 3

print('This video is', duration_in_deciseconds, 'deciseconds, which is', duration_in_minutes, 'minutes, which is', duration_in_frames, 'frames')
#print(Random_List)

This video is 39331 deciseconds, which is 65.55166666666666 minutes, which is 117993 frames


## Creating Array

## Time Integration (deciseconds) 

In [109]:
BehaviorTime = []

for index, value in enumerate(Random_List):
    BehaviorTime.append((value, index))

In [110]:
#print(BehaviorTime)

### Ensures you only analyze expected length of  Session 


length_of_session = (int(input("How long was the session in minutes?"))) * 600
print(length_of_session)
BehaviorTime = BehaviorTime[:(length_of_session+1)]

#### Removes all info after selected end time 

In [111]:
print('This video is ', len(BehaviorTime), ' deciseconds, which is', (len(BehaviorTime)/600), 'minutes' )
print()
#print(BehaviorTime)

This video is  39331  deciseconds, which is 65.55166666666666 minutes



## Adds Stim Count and Frame Rate to Array 

In [112]:
for z in BehaviorTime:
    behavior = z[0]
    time= z[1]
behavior_type_and_duration = []
current_behavior = BehaviorTime[0][0]
current_time = BehaviorTime[0][1]  
duration = 0
stim_count=0
frames = 0

for behavior, time in BehaviorTime:
    if behavior == current_behavior:
        start_time= current_time
        duration += 1
            
    else:
        if duration==1:
            end_time=start_time+1
        else:
            end_time= start_time + duration
        frames= duration*6
        stim_count= round(duration/6)
        behavior_type_and_duration.append((current_behavior, duration,frames,stim_count,start_time, end_time))
        current_behavior = behavior
        current_time = time
        start_time= time
        duration = 1
        end_time= time+1
        
# Append the final (current_behavior, current_time, count) tuple outside the loop
behavior_type_and_duration.append((current_behavior, duration, frames, stim_count, start_time, end_time))

# Make a copy of the original behavior_type_and_duration list
behaviortdt= list(behavior_type_and_duration)

In [113]:
#print(behaviortdt)
#print(len(behaviortdt))

# Check Test Behavior Counts CSV

In [114]:
# Initialize a dictionary to store total durations for each behavior label
total_durations = {label: 0 for label in behavior_labels.values()}

# Iterate through the behavior_type_and_duration list
for behavior, duration, _, _, _, _ in behavior_type_and_duration:
    # Get the behavior label based on the behavior code
    behavior_label = behavior_labels.get(behavior, "Unknown")
    
    # Add the duration to the total for the corresponding behavior label
    total_durations[behavior_label] += duration

# Print the total duration for each behavior label
for behavior_label, total_duration in total_durations.items():
    print(f"{behavior_label}: {total_duration}")



insignificant: 3713
Rear: 55
Investigate Type 1: 1215
Investigate Type 2: 12643
Contra-Itch: 1136
Investigate Type 3: 591
Contra-Body Groom: 224
Face Groom Type 1: 342
Dive/Scrunch: 341
Head Groom: 74
Ipsi-Orient: 2810
Face Groom Type 2: 4
Ipsi-Body Groom: 203
Ipsi-Itch Type 1: 237
Paw-Groom: 23
Locomotion: 13509
Contra-Orient: 2211


In [115]:
#length_of_session = (int(input("How long was the session in minutes?"))) * 600 
#print(length_of_session) 
behaviortdt = [tup for tup in behaviortdt if tup[5] <= 39001]

In [116]:
print('This video is ', behaviortdt[-1][-1], ' deciseconds, which is', (behaviortdt[-1][-1]/600), 'minutes' )
print()
#print(behaviortdt)

This video is  39001  deciseconds, which is 65.00166666666667 minutes



## Add Bout Count to the Array 

In [117]:
enumerated_behaviortdt = []
behavior_index = {}  # Dictionary to store behavior and its corresponding index

for behavior, duration, frames, stim_count, start_time, end_time in behaviortdt:
    if behavior not in behavior_index:
        behavior_index[behavior] = 1
    else:
        behavior_index[behavior] += 1
    
    bout = behavior_index[behavior]
    enumerated_behaviortdt.append((behavior,stim_count,frames, bout, duration,start_time,end_time))

print(enumerated_behaviortdt)

[(26, 0, 12, 1, 2, 0, 2), (28, 0, 6, 1, 1, 2, 3), (18, 0, 6, 1, 1, 3, 4), (28, 0, 6, 2, 1, 4, 5), (8, 0, 6, 1, 1, 5, 6), (28, 0, 6, 3, 1, 6, 7), (27, 0, 6, 1, 1, 7, 8), (26, 3, 96, 2, 16, 8, 24), (28, 0, 6, 4, 1, 24, 25), (8, 0, 6, 2, 1, 25, 26), (27, 0, 6, 2, 1, 26, 27), (18, 0, 18, 2, 3, 27, 30), (26, 0, 18, 3, 3, 30, 33), (8, 0, 12, 3, 2, 33, 35), (26, 0, 18, 4, 3, 35, 38), (8, 0, 6, 4, 1, 38, 39), (26, 0, 12, 5, 2, 39, 41), (8, 0, 12, 5, 2, 41, 43), (26, 0, 6, 6, 1, 43, 44), (8, 0, 12, 6, 2, 44, 46), (3, 0, 6, 1, 1, 46, 47), (8, 1, 48, 7, 8, 47, 55), (26, 0, 12, 7, 2, 55, 57), (18, 0, 6, 3, 1, 57, 58), (28, 0, 6, 5, 1, 58, 59), (15, 0, 6, 1, 1, 59, 60), (28, 0, 6, 6, 1, 60, 61), (7, 0, 6, 1, 1, 61, 62), (18, 0, 12, 4, 2, 62, 64), (15, 0, 6, 2, 1, 64, 65), (18, 0, 6, 5, 1, 65, 66), (26, 0, 6, 8, 1, 66, 67), (18, 0, 18, 6, 3, 67, 70), (26, 0, 12, 9, 2, 70, 72), (8, 0, 6, 8, 1, 72, 73), (26, 3, 114, 10, 19, 73, 92), (8, 0, 6, 9, 1, 92, 93), (18, 0, 12, 7, 2, 93, 95), (26, 1, 36, 11, 6

# Stim Count Calcultaion

In [118]:
#BCBDT = []

# Calculate durations and update BCBDT
#for behavior, duration, frames, bout, start_time, end_time in enumerated_behaviortdt:
    #stim_count = round(duration / 3)
    #if stim_count < 1:
        #stim_count = 0
    #if stim_count > 0:
        #BCBDT.append((behavior, duration, frames, bout, stim_count, start_time, end_time))

# Print the modified BCBDT
#print("Modified BCBDT:", BCBDT)

# Assign enumerated_behaviortdt to BCBDST
BCBDST = enumerated_behaviortdt


In [119]:
behavior_labels = {
    0.0: "insignificant",
    1.0: "insignificant",
    2.0: "insignificant",
    3.0: "insignificant",
    4.0: "Rear",
    5.0: "insignificant",
    6.0: "insignificant",
    7.0: "Investigate Type 1",
    8.0: "Investigate Type 2", 
    9.0: "Contra-Itch",
    10.0: "Investigate Type 3",
    11.0: "insignificant",
    12.0: "insignificant",
    13.0: "insignificant",
    14.0: "Contra-Body Groom",
    15.0: "Face Groom Type 1", 
    16.0: "Dive/Scrunch",
    17.0: "Head Groom",
    18.0: "Ipsi-Orient",
    19.0: "insignificant",
    20.0: "Face Groom Type 2",
    21.0: "Ipsi-Body Groom",
    22.0: "Ipsi-Itch Type 1",
    23.0: "insignificant",
    24.0: "insignificant",
    25.0: "Paw-Groom",
    26.0: "Locomotion",
    27.0: "insignificant",
    28.0: "Contra-Orient",
}

bdbt_dict = {}  # Dictionary to store lists of behaviors

for behavior,stim_count,frames, bout, duration,start_time,end_time in BCBDST:
    behavior_label = behavior_labels.get(behavior, "Unknown Behavior")
    if behavior_label not in bdbt_dict:
        bdbt_dict[behavior_label] = []
    bdbt_dict[behavior_label].append((behavior, stim_count, frames, bout, duration, start_time,end_time))


In [120]:


#print(bdbt_dict)

In [121]:
keys_list = bdbt_dict.keys()
print(keys_list)

dict_keys(['Locomotion', 'Contra-Orient', 'Ipsi-Orient', 'Investigate Type 2', 'insignificant', 'Face Groom Type 1', 'Investigate Type 1', 'Contra-Itch', 'Dive/Scrunch', 'Ipsi-Itch Type 1', 'Investigate Type 3', 'Ipsi-Body Groom', 'Paw-Groom', 'Head Groom', 'Contra-Body Groom', 'Rear', 'Face Groom Type 2'])


## Behavior of Interest

In [122]:
for file_path in infiles: # if taking from Z Drive 
    folders = (os.path.dirname(file_path)).split('/')
    print(folders)
    print(len(folders))
    # Check if there are at least 5 elements in the 'folders' list
    if len(folders) >= 5:
        fol3 = folders[3].split('-')  # Split element at position 3 by hyphen
        fol4 = folders[4].split('-')  # Split element at position 4 by hyphen
        
        print("Folder 3:", fol3)
        print("Folder 4:", fol4)
    else:
        print("The folder path does not have enough elements to access positions 3 and 4.")
    

['Z:', 'KayCei', 'HDC', 'AH1-KO', '120423-LIGHTS OFF']
5
Folder 3: ['AH1', 'KO']
Folder 4: ['120423', 'LIGHTS OFF']


## Stim Count and Average Duration 

In [123]:
from tabulate import tabulate
import matplotlib.pyplot as plt
import csv


def round_float(value):
    return round(value, 3)

average_duration_by_key = {}

for key, tuples_list in bdbt_dict.items():
    #if key=='insignificant':
        #continue #skip "insignificant" key and its data
    durations = [tuple_item[4] for tuple_item in tuples_list]
    average_duration = (sum(durations) / len(durations))/10
    average_duration_by_key[key] =average_duration

table_data = [(key, avg_duration) for key, avg_duration in average_duration_by_key.items()]
headers = ["Key", "Average Duration(seconds)"]

table = tabulate(table_data, headers, tablefmt="pretty")
print(table)

# Create a dictionary for the CSV data
csv_data = {f'{fol3[0]} {fol4[1]} Duration': average_duration_by_key}

# Plot the data
plt.figure(figsize=(10, 6))
plt.bar(average_duration_by_key.keys(), average_duration_by_key.values())
plt.xlabel("Key")
plt.ylabel("Average Duration (seconds)")
plt.title(f"Average Durations by Key {fol3[0]} {fol4[1]}")
plt.xticks(rotation=45, ha='right')
plt.tight_layout()

## Check if the user selected any files
if infiles:
    # Assuming infiles is a list of selected file paths, let's use the first one
    selected_file = infiles[0]

    # Extract the directory path from the selected file's full path
    directory_path = os.path.dirname(selected_file)

    # Save the plot in the same directory as the selected file with the specified filename
    filename = f'{fol3[0]} {fol4[1]} Duration.png'
    save_path = os.path.join(directory_path, filename)
    plt.savefig(save_path)
    print(f"Plot saved in: {save_path}")

    # Save the CSV data to a CSV file
    csv_filename = f'{fol3[0]} {fol4[1]} Duration.csv'
        # Construct the full path for saving the CSV in the same directory
    csv_save_path = os.path.join(directory_path, csv_filename)

    # Save the CSV data to the CSV file
    with open(csv_save_path, mode="w", newline="") as csv_file:
        csv_writer = csv.writer(csv_file)
        for dictionary_name, data_dict in csv_data.items():
            csv_writer.writerow([dictionary_name])  # Write the dictionary name as a header
            csv_writer.writerow(["Key", "Average Duration (seconds)"])
            for key, avg_duration in data_dict.items():
                csv_writer.writerow([key, avg_duration])

    print(f"CSV data saved in: {csv_save_path}")
    

+--------------------+---------------------------+
|        Key         | Average Duration(seconds) |
+--------------------+---------------------------+
|     Locomotion     |    0.3249094858797973     |
|   Contra-Orient    |    0.1271345029239766     |
|    Ipsi-Orient     |     0.136678032148076     |
| Investigate Type 2 |    0.27811874583055374    |
|   insignificant    |    0.13977666538313438    |
| Face Groom Type 1  |    0.18688524590163935    |
| Investigate Type 1 |    0.24059405940594059    |
|    Contra-Itch     |    0.14253164556962025    |
|    Dive/Scrunch    |    0.12266187050359711    |
|  Ipsi-Itch Type 1  |    0.10308370044052864    |
| Investigate Type 3 |    0.21032028469750888    |
|  Ipsi-Body Groom   |    0.13605442176870747    |
|     Paw-Groom      |          0.14375          |
|     Head Groom     |    0.20555555555555555    |
| Contra-Body Groom  |    0.16641221374045803    |
|        Rear        |    0.15714285714285714    |
| Face Groom Type 2  |    0.133

In [124]:
from tabulate import tabulate
import matplotlib.pyplot as plt

In [125]:
total_count_by_key = {}

for key, tuples_list in bdbt_dict.items():
    #if key=='insignificant':
        #continue #skip "insignificant" key and its data
    durations = [tuple_item[4] for tuple_item in tuples_list]
    behavior= [tuple_item[0] for tuple_item in tuples_list]
    average_count = sum(durations)
    
    total_count_by_key[key] = average_count
    
     # Display the result for each key inside the loop
    print(f"Key: {key}, Average Count: {average_count}")

table_data = [(key, avg_count) for key, avg_count in total_count_by_key.items()]
headers = ["Key", "Average Count"]


table = tabulate(table_data, headers, tablefmt="pretty")
print(table)

Key: Locomotion, Average Count: 13461
Key: Contra-Orient, Average Count: 2174
Key: Ipsi-Orient, Average Count: 2806
Key: Investigate Type 2, Average Count: 12507
Key: insignificant, Average Count: 3630
Key: Face Groom Type 1, Average Count: 342
Key: Investigate Type 1, Average Count: 1215
Key: Contra-Itch, Average Count: 1126
Key: Dive/Scrunch, Average Count: 341
Key: Ipsi-Itch Type 1, Average Count: 234
Key: Investigate Type 3, Average Count: 591
Key: Ipsi-Body Groom, Average Count: 200
Key: Paw-Groom, Average Count: 23
Key: Head Groom, Average Count: 74
Key: Contra-Body Groom, Average Count: 218
Key: Rear, Average Count: 55
Key: Face Groom Type 2, Average Count: 4
+--------------------+---------------+
|        Key         | Average Count |
+--------------------+---------------+
|     Locomotion     |     13461     |
|   Contra-Orient    |     2174      |
|    Ipsi-Orient     |     2806      |
| Investigate Type 2 |     12507     |
|   insignificant    |     3630      |
| Face Groom 

##  Number of Bouts

In [126]:
bout_by_key = {}

for key, tuples_list in bdbt_dict.items():
    #if key=='insignificant':
        #continue #skip "insignificant" key and its data
    bouts = [tuple_item[3] for tuple_item in tuples_list]
    sum_of_bouts = len(bouts)
    bout_by_key[key] = sum_of_bouts

table_data = [(key, sum_value) for key, sum_value in bout_by_key.items()]
headers = ["Key", "Total Bouts"]

table = tabulate(table_data, headers, tablefmt="pretty")
print(table)

# Create a dictionary for the CSV data
csv_data_bouts = {f'{fol3[0]} {fol4[1]}': bout_by_key}
# Plot the data
plt.figure(figsize=(10, 6))
plt.bar(bout_by_key.keys(), bout_by_key.values())
plt.xlabel("Key")
plt.ylabel("Total Bouts")
plt.title(f'Total Bouts by Key {fol3[0]} {fol4[1]}')
plt.xticks(rotation=45, ha='right')  # Set the x-axis ticks and labels
plt.tight_layout()
# Check if the user selected any files
if infiles:
    # Assuming infiles is a list of selected file paths, let's use the first one
    selected_file = infiles[0]

    # Extract the directory path from the selected file's full path
    directory_path = os.path.dirname(selected_file)
    # Save the plot as "(specific animals) Bout.png"
    filename_bout= f'{fol3[0]} {fol4[1]} Bouts.png'
    save_path= os.path.join(directory_path,filename_bout)
    plt.savefig(save_path)

    # Save the CSV data to a CSV file
    csv_filename_bouts = f'{fol3[0]} {fol4[1]} Bouts.csv'
    csv_save_path = os.path.join(directory_path, csv_filename_bouts)
    
    with open(csv_save_path, mode="w", newline="") as csv_file:
        csv_writer = csv.writer(csv_file)
        for dictionary_name, data_dict in csv_data_bouts.items():
            csv_writer.writerow([dictionary_name])  # Write the dictionary name as a header
            csv_writer.writerow(["Key", "Total Bouts"])
            for key, sum_value in data_dict.items():
                csv_writer.writerow([key, sum_value])
    print(f"CSV data saved in: {csv_save_path}")

+--------------------+-------------+
|        Key         | Total Bouts |
+--------------------+-------------+
|     Locomotion     |    4143     |
|   Contra-Orient    |    1710     |
|    Ipsi-Orient     |    2053     |
| Investigate Type 2 |    4497     |
|   insignificant    |    2597     |
| Face Groom Type 1  |     183     |
| Investigate Type 1 |     505     |
|    Contra-Itch     |     790     |
|    Dive/Scrunch    |     278     |
|  Ipsi-Itch Type 1  |     227     |
| Investigate Type 3 |     281     |
|  Ipsi-Body Groom   |     147     |
|     Paw-Groom      |     16      |
|     Head Groom     |     36      |
| Contra-Body Groom  |     131     |
|        Rear        |     35      |
| Face Groom Type 2  |      3      |
+--------------------+-------------+
CSV data saved in: Z:/KayCei/HDC/AH1-KO/120423-LIGHTS OFF\AH1 LIGHTS OFF Bouts.csv


# Check!!!

In [127]:
print(csv_save_path)

Z:/KayCei/HDC/AH1-KO/120423-LIGHTS OFF\AH1 LIGHTS OFF Bouts.csv


In [128]:
for file_path in infiles:  # if taking from Z Drive 
    folders = os.path.dirname(file_path).rsplit('\\', 1)
    print(folders)

['Z:/KayCei/HDC/AH1-KO/120423-LIGHTS OFF']


In [129]:
import pandas as pd

# Define the file paths for the two CSV files and the output CSV file
bout_csv_filename = os.path.join('/'.join(folders), csv_filename_bouts)
duration_csv_filename = os.path.join('/'.join(folders), csv_filename )

# Read the CSV files into DataFrames
bout_df = pd.read_csv(bout_csv_filename, skiprows=[0])
duration_df = pd.read_csv(duration_csv_filename, skiprows=[0])

print("Columns in bout_df:", bout_df.columns)
print("Columns in duration_df:", duration_df.columns)

Columns in bout_df: Index(['Key', 'Total Bouts'], dtype='object')
Columns in duration_df: Index(['Key', 'Average Duration (seconds)'], dtype='object')


In [130]:
# Merge the two DataFrames on the shared key column (e.g., "Key")
merged_df = bout_df.merge(duration_df, on="Key", how="inner")

# Perform the multiplication: Total Bouts * Average Duration (seconds)
merged_df["Result(seconds)"] = merged_df["Total Bouts"] * merged_df["Average Duration (seconds)"]

# Display the results DataFrame within the Jupyter Notebook
display(merged_df)


,Key,Total Bouts,Average Duration (seconds),Result(seconds)
0,Locomotion,4143,0.324909,1346.1
1,Contra-Orient,1710,0.127135,217.4
2,Ipsi-Orient,2053,0.136678,280.6
3,Investigate Type 2,4497,0.278119,1250.7
4,insignificant,2597,0.139777,363.0
5,Face Groom Type 1,183,0.186885,34.2
6,Investigate Type 1,505,0.240594,121.5
7,Contra-Itch,790,0.142532,112.6
8,Dive/Scrunch,278,0.122662,34.1
9,Ipsi-Itch Type 1,227,0.103084,23.4


In [131]:
# Calculate the sum of the "Result" column in the DataFrame
total_result = (merged_df["Result(seconds)"].sum())/60

print(f"Sum of the 'Result' column: {total_result} minutes")


Sum of the 'Result' column: 65.00166666666665 minutes


## Creating CSV

In [132]:
rounded_dict = {}

for key, value_list in bdbt_dict.items():
    rounded_values = [
        (
            round(v[0]),     # Rounding and converting to integer
            v[1], v[2], v[3], 
            round(v[4], 3),
            round(v[5], 2),  
            round(v[6], 2)   
        )
        for v in value_list
    ]
    rounded_dict[key] = rounded_values

#print(rounded_dict)

bdbt_dict=rounded_dict
#print(bdbt_dict)

In [133]:
from json import dumps
string_bdbt_dict=dumps(bdbt_dict)

In [134]:
# Find the start index of the first '{' character
start_index = string_bdbt_dict.index('{')

# Find the end index of the last '}' character
end_index = string_bdbt_dict.rindex('}') + 1

# Extract the dictionary portion of the string
dict_str = string_bdbt_dict[start_index:end_index]

# Find the start indices of key-value pairs using '":'
key_value_end = []
for pos, char in enumerate(dict_str):
    if dict_str[pos:pos+3] == '": ':
        key_value_end.append(pos)
        
#print(key_value_end)      

# Initialize lists to store extracted keys and values
keys = []
values = []
value_ends=[]

# Iterate through the start indices of key-value pairs
for key_value_start in key_value_end:
    # Find the start index of the value by searching for '[[' after the key-value pair
    value_start = dict_str.find('[[', key_value_start) + 1
    
    # Find the end index of the value by searching for ']]' after the value_start
    value_end = dict_str.find(']]', value_start)+ 2
    value_ends.append(value_end)

    # Extract the value using the calculated start and end indices
    value = dict_str[value_start:value_end]
    values.append(value)
    # Extract the key using the portion of the string before the '":'
    if key_value_start == key_value_end[0]:
        key = dict_str[2:key_value_start].strip()
        keys.append(key)
    else:
        key = dict_str[value_ends[-2]+ 3:key_value_start].strip()
        keys.append(key)
        
#print(values)
#print(value_ends)
modified_values = {}  # Initialize an empty dictionary to store modified values

# Iterate through the keys and values
for key, value in zip(keys, values):
    modified_value = value.replace(" ", "").replace("],", "]").replace("]]","]")
    modified_values[key] = modified_value

In [135]:
keys_to_write = ["Rear", "Investigate Type 1", "Investigate Type 2",
                 "Contra-Itch", "Investigate Type 3", "Contra-Body Groom",
                 "Face Groom Type 1", "Dive/Scrunch", "Head Groom", "Ipsi-Orient",
                 "Face Groom Type 2", "Ipsi-Body Groom", "Ipsi-Itch Type 1",
                 "Paw-Groom", "Locomotion", "Contra-Orient"]

In [136]:
# Create a dictionary to hold cleaned and possibly split values
cleaned_values = {}

In [137]:
# Iterate through the keys and values, cleaning and splitting if needed
for key in keys_to_write:
    value = modified_values[key].strip()
    
    if len(value) > 32500:
        # Find the last "]" before 32500 characters
        split_index = value.rfind("]", 0, 32500)
        
        if split_index != -1:
            # Split the value into two parts
            part1 = value[:split_index + 1]  # Include the last "]"
            part2 = value[split_index + 1:]
            
            cleaned_values[key] = part1  # Store the first part
            
            continue_key = f"{key} Continue"
            continue_count = 1
            
            while len(part2) > 32500:
                continue_count += 1
                split_index = part2.rfind("]", 0, 32500)  # Find the last "]" in the remaining portion
                part1 = part2[:split_index + 1]
                part2 = part2[split_index + 1:]
                cleaned_values[f"{key} Continue {continue_count}"] = part1
            cleaned_values[f"{key} Continue {continue_count + 1}"] = part2  # Store the remaining part
            
        else:
            # If "]" is not found, just store the value as it is
            cleaned_values[key] = value
    else:
        cleaned_values[key] = value


In [138]:
# Format specific values in cleaned_values dictionary
formatted_values = {'"{}"'.format(key): value for key, value in cleaned_values.items()}

In [139]:
# Initialize the list of behavior keys you want to include in mouse_data
behavior_keys = list(cleaned_values.keys())

In [140]:
print(behavior_keys)

['Rear', 'Investigate Type 1', 'Investigate Type 2', 'Investigate Type 2 Continue 2', 'Investigate Type 2 Continue 3', 'Investigate Type 2 Continue 4', 'Contra-Itch', 'Investigate Type 3', 'Contra-Body Groom', 'Face Groom Type 1', 'Dive/Scrunch', 'Head Groom', 'Ipsi-Orient', 'Ipsi-Orient Continue 2', 'Face Groom Type 2', 'Ipsi-Body Groom', 'Ipsi-Itch Type 1', 'Paw-Groom', 'Locomotion', 'Locomotion Continue 2', 'Locomotion Continue 3', 'Locomotion Continue 4', 'Contra-Orient', 'Contra-Orient Continue 2']


In [141]:
# Specify the output path for the CSV file
output_path = r"Z:\KayCei\Practice 16.csv"

data_dict = {}
session_numbers = {}

# Iterate through each file in infiles
for file_path in infiles: # if taking from Z Drive 
    folders = (os.path.dirname(file_path)).split('/')
    print(folders)
    # Extract the mouse name from the file path
    mouse_name = fol3[0]
    # Extract the genotype from the file path
    genotype = fol3[1]
    # Extract the date from the file path
    date = fol4[0]
    environment= fol4[1]
#for file_path in infiles:   #if taking files from D drive
    #folders = (os.path.dirname(file_path)).split('/')
    #print(folders)
    # Extract the mouse name from the file path
    #mouse_name = folders[6].split('-')[0]
    # Extract the genotype from the file path
    #genotype = folders[6].split('-')[1]
    # Extract the date from the file path
    #date = folders[-1].split('-')[0]    
# Determine the session number for this mouse and stim behavior
    key = (mouse_name, environment)
    if key not in session_numbers:
        session_numbers[key] = 1
    else:
        session_numbers[key] += 1
    session_number = session_numbers[key]

   # Initialize the mouse_data dictionary
    mouse_data = {
        "Mouse": mouse_name,
        "Genotype": genotype,
        "Date": date,
        "Environment": environment,
        "Session Number": session_number,
    }

    for behavior_key in behavior_keys:
        cleaned_value = cleaned_values.get(behavior_key, 0)
        mouse_data[behavior_key] = cleaned_value

        if not behavior_key.endswith("Continue"):
            bout_count = bout_by_key.get(behavior_key, 0)
            average_duration = average_duration_by_key.get(behavior_key, 0)

            mouse_data[f"{behavior_key} Total Bout Count"] = bout_count
            mouse_data[f"{behavior_key} Average Duration(s)"] = average_duration

            # Check if the behavior_key ends with "Continue"
            if behavior_key.endswith("Continue"):
                # Remove the "Total bout" and "Average duration" columns
                del mouse_data[f"{behavior_key} Total Bout Count"]
                del mouse_data[f"{behavior_key} Average Duration(s)"]

    # Add mouse_data to data_dict
    data_dict[mouse_name] = mouse_data

['Z:', 'KayCei', 'HDC', 'AH1-KO', '120423-LIGHTS OFF']


In [142]:
import re

# Define a pattern to match "Continue" followed by one or more digits
pattern = r"Continue \d+ (Total Bout Count|Average Duration\(s\))"

# Create a list of keys to remove based on the pattern
keys_to_remove = [key for key in mouse_data.keys() if re.search(pattern, key)]

# Remove the keys and their associated values from the dictionary
for key in keys_to_remove:
    del mouse_data[key]

# Print the updated dictionary
#print(mouse_data)

In [143]:
#print(keys_to_remove)

In [144]:
# Using a list comprehension
#keys = [key for key in mouse_data.keys()]
#print(keys)

In [145]:
# Define the headers for the CSV file
headers = list(mouse_data.keys())

In [146]:
print(headers)

['Mouse', 'Genotype', 'Date', 'Environment', 'Session Number', 'Rear', 'Rear Total Bout Count', 'Rear Average Duration(s)', 'Investigate Type 1', 'Investigate Type 1 Total Bout Count', 'Investigate Type 1 Average Duration(s)', 'Investigate Type 2', 'Investigate Type 2 Total Bout Count', 'Investigate Type 2 Average Duration(s)', 'Investigate Type 2 Continue 2', 'Investigate Type 2 Continue 3', 'Investigate Type 2 Continue 4', 'Contra-Itch', 'Contra-Itch Total Bout Count', 'Contra-Itch Average Duration(s)', 'Investigate Type 3', 'Investigate Type 3 Total Bout Count', 'Investigate Type 3 Average Duration(s)', 'Contra-Body Groom', 'Contra-Body Groom Total Bout Count', 'Contra-Body Groom Average Duration(s)', 'Face Groom Type 1', 'Face Groom Type 1 Total Bout Count', 'Face Groom Type 1 Average Duration(s)', 'Dive/Scrunch', 'Dive/Scrunch Total Bout Count', 'Dive/Scrunch Average Duration(s)', 'Head Groom', 'Head Groom Total Bout Count', 'Head Groom Average Duration(s)', 'Ipsi-Orient', 'Ipsi-O

In [147]:
if os.path.exists(output_path):
    # Keep track of existing entries in the CSV to avoid duplicates
    existing_entries = set()
    entry_key2_session_numbers = {}
    sorted_entries = []

    # Read existing data from the CSV and add it to the set
    with open(output_path, 'r') as csvfile:
        reader = csv.DictReader(csvfile, delimiter=';')
        for row in reader:
            entry_key = (row["Mouse"], row["Date"], row["Environment"])
            entry_key2 = (row["Mouse"], row["Date"], row["Environment"], row["Genotype"])
            existing_entries.add(entry_key)
            entry_key2_session_numbers[entry_key2] = int(row["Session Number"])
            sorted_entries.append(row)
        
else:
    existing_entries = set()
    entry_key2_session_numbers = {}
    
# Ensure the output directory exists before writing the CSV file
output_dir = os.path.dirname(output_path)
os.makedirs(output_dir, exist_ok=True)

# Write the data from data_dict to the CSV file (in append mode 'a')
with open(output_path, 'a', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=headers, delimiter=';')

    # Check if the file is empty, if so, write the header
    if csvfile.tell() == 0:
        writer.writeheader()

for mouse_data in data_dict.values():
    entry_key = (mouse_data["Mouse"], mouse_data["Date"], mouse_data["Environment"])
    entry_key2 = (mouse_data["Mouse"], mouse_data["Date"], mouse_data["Environment"])

    # Check if the entry already exists in the CSV
    if entry_key not in existing_entries:
        session_number = entry_key2_session_numbers.get(entry_key2, 0) + 1
        existing_entries.add(entry_key)
        entry_key2_session_numbers[entry_key2] = session_number

        for formatted_key in behavior_keys:
            if formatted_key in formatted_values:
                mouse_data[formatted_key] = formatted_values[formatted_key]

        # Write the complete mouse data to the CSV
        sorted_entries.append(mouse_data)  # Add the new entry

        print("Entry added Successfully:", entry_key)
    else:
        print("Entry already exists in the CSV:", entry_key)

Entry added Successfully: ('AH1', '120423', 'LIGHTS OFF')


In [148]:
# Add unique keys from unique_entry_keys to fieldnames
fieldnames=headers

In [149]:
# Convert the fieldnames list to a set
fieldnames_set = set(headers)  # Replace 'headers' with your actual fieldnames list

# Extract the keys from the dictionaries within sorted_entries and create a set
entry_keys_set = set()
for entry in sorted_entries:
    entry_keys_set.update(entry.keys())

# Find the unique keys in fieldnames_set and entry_keys_set
unique_fieldnames = fieldnames_set.difference(entry_keys_set)
unique_entry_keys = entry_keys_set.difference(fieldnames_set)

# Print the unique keys
print("Unique keys in fieldnames_set:", unique_fieldnames)
print("Unique keys in entry_keys_set:", unique_entry_keys)


Unique keys in fieldnames_set: set()
Unique keys in entry_keys_set: set()


In [150]:
for unique_entry_key in unique_entry_keys:
    # Check if unique_entry_key exists (not None and not an empty string)
    if unique_entry_key:
        # Extract the prefix by splitting at the first space followed by a digit
        prefix_to_match = re.split(r'\s+\d', unique_entry_key)[0]

        # Find the position to insert unique_entry_key
        insert_position = None

        for i in range(len(fieldnames) - 1, -1, -1):
            if fieldnames[i].startswith(prefix_to_match):
                insert_position = i + 1
                break

        # If the prefix is not found, append the key to the end
        if insert_position is None:
            insert_position = len(fieldnames)

        # Insert the unique_entry_key at the specified position
        fieldnames.insert(insert_position, unique_entry_key)
    else:
        # Skip to the next iteration (next unique_entry_key)
        continue

In [151]:
print(fieldnames)

['Mouse', 'Genotype', 'Date', 'Environment', 'Session Number', 'Rear', 'Rear Total Bout Count', 'Rear Average Duration(s)', 'Investigate Type 1', 'Investigate Type 1 Total Bout Count', 'Investigate Type 1 Average Duration(s)', 'Investigate Type 2', 'Investigate Type 2 Total Bout Count', 'Investigate Type 2 Average Duration(s)', 'Investigate Type 2 Continue 2', 'Investigate Type 2 Continue 3', 'Investigate Type 2 Continue 4', 'Contra-Itch', 'Contra-Itch Total Bout Count', 'Contra-Itch Average Duration(s)', 'Investigate Type 3', 'Investigate Type 3 Total Bout Count', 'Investigate Type 3 Average Duration(s)', 'Contra-Body Groom', 'Contra-Body Groom Total Bout Count', 'Contra-Body Groom Average Duration(s)', 'Face Groom Type 1', 'Face Groom Type 1 Total Bout Count', 'Face Groom Type 1 Average Duration(s)', 'Dive/Scrunch', 'Dive/Scrunch Total Bout Count', 'Dive/Scrunch Average Duration(s)', 'Head Groom', 'Head Groom Total Bout Count', 'Head Groom Average Duration(s)', 'Ipsi-Orient', 'Ipsi-O

In [152]:
# Sort the entries based on date within each group of entries that share the same entry_key2
sorted_entries.sort(key=lambda x: (x["Mouse"], x["Date"], x["Environment"]))

# Update the session numbers based on the sorted order
entry_key2_counter = {}  # To track the session numbers within each entry_key2 group

for entry in sorted_entries:
    entry_key2 = (entry["Mouse"], entry["Date"], entry["Environment"], entry["Genotype"])
    
    if entry_key2 not in entry_key2_counter:
        entry_key2_counter[entry_key2] = 1
    else:
        entry_key2_counter[entry_key2] += 1
    
    entry["Session Number"] = entry_key2_counter[entry_key2]

# Sort the entries based on date within each group of entries that share the same entry_key2
sorted_entries.sort(key=lambda x: (x["Mouse"], x["Date"], x["Environment"], x["Genotype"]))

# Rewrite the CSV file with sorted and formatted entries, including any missing fields
with open(output_path, 'w', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames, delimiter=';')
    writer.writeheader()
    writer.writerows(sorted_entries)
